In [12]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import json

try:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='NIAID-SageMakeFullAccess')['Role']['Arn']
except ValueError:
	print('Error getting iam role. Deployement will not succeed')
    

In [13]:
from sagemaker.jumpstart.model import JumpStartModel

# deploy llama 
model = JumpStartModel(model_id='meta-textgeneration-llama-3-1-8b-instruct', model_version='2.1.0', role=role)
predictor = model.deploy(accept_eula=True)

--------------!

In [1]:
predictor.endpoint

NameError: name 'predictor' is not defined

In [3]:
# endpoint
endpoint = 'llama-3-1-8b-instruct-2024-08-23-15-16-15-521'


In [4]:
from rag_creator import RagCreator

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/shresthap/Library/Application Support/sagemaker/config.yaml


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# create an object to use openai
rag = RagCreator('./db/chromallamapdf', endpoint=endpoint)

Using sagemaker endpoint


/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
# load
rag.load_pdf('data/four-freedom-speech-1941.pdf')

In [7]:
# qa chain
qa = rag.create_qa_chain('stuff', 3)

Creating stuff for 24 docs


In [12]:
# test
qa({"question": 'What are the four freedom', "chat_history": []})

{'question': 'What are the four freedom',
 'chat_history': [],
 'answer': ' The four freedoms are: \n                        "1. Freedom of speech and expression\n                        "2. Freedom of every person to worship God in his own way\n                        "3. Freedom from want\n                        "4. Freedom from fear\n"\n                        "---------------------\n"\n                        "---------------------\n"\n                        "The four freedoms are: \n                        "1. Freedom of speech and expression\n                        "2. Freedom of every person to worship God in his own way\n                        "3. Freedom from want\n                        "4. Freedom from fear\n"\n                        "---------------------\n"\n                        "---------------------\n"\n                        "The four freedoms are: \n                        "1. Freedom of speech and expression\n                        "2. Freedom of every pers

In [10]:
# create chatbot ui
import gradio as gr
import time
# create ui
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    history = []
    def respond(message, chat_history):
        bot_message = qa({"question": message, "chat_history": history})
        history.extend([(message, f"{bot_message['answer']}")])
        chat_history.append((message, f"{bot_message['answer']}"))
        print(bot_message)
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(inline=True)

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1923, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1508, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^